<a href="https://colab.research.google.com/github/mengmomengshang/TSDT/blob/main/Easy_Wav2Lip_v8.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to my Easy Wav2Lip colab!

My goal is to make lipsyncing with this tool easy, fast and great looking!

Please view the GitHub for instructions: [https://github.com/anothermartz/Easy-Wav2Lip](https://github.com/anothermartz/Easy-Wav2Lip?tab=readme-ov-file#best-practices)

In [14]:
version = 'v8.3'
#@title <h1>Step 1: Setup "Easy-Wav2Lip"</h1> With one button: it's really that easy!
#@markdown 👈 Click that little circle play button first - it will ask for Google Drive access: <br>
#@markdown > Accept if your files are on Google Drive (recommended).
#@markdown <br> Alternatively, you can click deny and upload files manually, but this is slower.

# ============== 修复关键：提前创建 basicsr 目录 ==============
!mkdir -p /usr/local/lib/python3.10/dist-packages/basicsr/data
!mkdir -p /content/Easy-Wav2Lip/checkpoints
!wget https://github.com/anothermartz/Easy-Wav2Lip/releases/download/v8.3/predictor.pkl -O /content/Easy-Wav2Lip/checkpoints/predictor.pkl
!wget https://github.com/YudongGuo/AD-NeRF/raw/master/data_util/face_parsing/checkpoints/79999_iter.pth -O /content/Easy-Wav2Lip/checkpoints/face_segmentation.pth

#check if already installed
import os
import sys
if os.path.exists('installed.txt'):
    os.remove('installed.txt')
if os.path.exists('last_file.txt'):
    os.remove('last_file.txt')
#check GPU is enabled
print('checking for GPU')
import torch
if not torch.cuda.is_available():
  sys.exit('No GPU in runtime. Please go to the "Runtime" menu, "Change runtime type" and select "GPU".')

#prompt to mount google drive
print('requesting Google Drive access')
try:
  from google.colab import drive
  drive.mount('/content/drive')
except:
  print("google drive not linked")


#start timer
import time
start_time = time.time()

#clone git
giturl = 'https://github.com/anothermartz/Easy-Wav2Lip.git'


!git clone -b {version} {giturl}
%cd 'Easy-Wav2Lip'
working_directory = os.getcwd()
!mkdir 'face_alignment' 'temp'

#install prerequisites
print('installing batch_face')
import warnings
warnings.filterwarnings("ignore", category=UserWarning,
                        module='torchvision.transforms.functional_tensor')
!pip install batch_face --quiet
!pip install basicsr==1.4.2 --quiet

# ============== 修复关键：确保复制成功 ==============
print('fixing basicsr degradations.py')
# 1. 确认源文件存在
if not os.path.exists('/content/Easy-Wav2Lip/degradations.py'):
  !cp /content/Easy-Wav2Lip/Easy-Wav2Lip/degradations.py /content/Easy-Wav2Lip/

# 2. 复制到目标位置
!cp /content/Easy-Wav2Lip/degradations.py /usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py

# ============== 新增：验证复制是否成功 ==============
if not os.path.exists('/usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py'):
  print("⚠️ 修复失败！手动创建链接")
  !ln -s /content/Easy-Wav2Lip/degradations.py /usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py

print('installing gfpgan')
!pip install gfpgan --quiet

!python install.py

from IPython.display import clear_output
clear_output()
print("✅ Installation complete, move to Step 2!")

# ============== 新增：创建完成标记 ==============
!touch installed.txt
with open('last_file.txt', 'w') as file:
    file.write(version)

#end timer
elapsed_time = time.time() - start_time
from easy_functions import format_time
print(f"Execution time: {format_time(elapsed_time)}")

✅ Installation complete, move to Step 2!
Execution time: 43s


In [15]:
import os
import sys

if not os.path.exists('installed.txt'):
  sys.exit('Step 1 has not been run in this instance! Please run step 1 each time you disconnect from a runtime.')

# ========== 修复 torchvision 兼容性问题 ==========
def fix_degradations_py():
    degradations_path = "/usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py"
    if not os.path.exists(degradations_path):
        print("⚠️ degradations.py 不存在，跳过修复")
        return

    with open(degradations_path, "r") as f:
        content = f.read()

    # 检查是否已经修复
    if "from torchvision.transforms.functional import rgb_to_grayscale" in content:
        print("✅ degradations.py 已修复")
        return

    # 进行修复
    new_content = content.replace(
        "from torchvision.transforms.functional_tensor import rgb_to_grayscale",
        """try:
    # 新版本 PyTorch
    from torchvision.transforms.functional import rgb_to_grayscale
except ImportError:
    # 旧版本 PyTorch
    from torchvision.transforms.functional_tensor import rgb_to_grayscale"""
    )

    with open(degradations_path, "w") as f:
        f.write(new_content)
    print("✅ 已修复 degradations.py")

# 执行修复
fix_degradations_py()
# ========== 修复结束 ==========

time
############################## user inputs #####################################
#@markdown <h1>Step 2: Select inputs:</h1>

# @markdown On destktop: <h1></h1>Click the folder icon ( 📁 ) at the left edge of colab, find your file, right click, copy path, paste it below:
#@markdown<br></br>
# @markdown On mobile: <h1></h1>Tap the hamburger button ( ☰ ) at the top left, click show file browser, find your file, long press on it, copy path, paste below:
video_file = "/content/视频剪辑25_爱给网_aigei_com.mp4" #@param {type:"string"}
vocal_file = "/content/音频转码_爱给网_aigei_com.wav" #@param {type:"string"}

#@markdown > Keep vocal_file blank if your video already has the desired speech audio encoded into it.
#@markdown # Quality
quality = "Enhanced" # @param ["Fast", "Improved", "Enhanced"]
#@markdown * <b><u>Fast</u></b>: Wav2Lip <br>
#@markdown * <b><u>Improved</u></b>: Wav2Lip with a feathered mask around the mouth to remove the square around the face <br>
#@markdown * <b><u>Enhanced</u></b>: Wav2Lip + mask + GFPGAN upscaling done on the face
#preview_quality = False #@param {type:"boolean"} - coming soon!
output_height = "full resolution" #@param ["half resolution", "full resolution", "480"] {allow-input: true}
use_previous_tracking_data = True #@param {type:"boolean"}
#@markdown Speeds up processing of the same video used multiple times - it should delete the last tracking file automatically when the video is changed but if it's failing after the first video, untick this box.

#@markdown
#------------------------------*Step 3*----------------------------------------!
#@markdown <h1>👈 Step 3:  Click the little circle play button on this cell! </h1> (Or press ctrl + F10) - Then wait for processing to complete.
# scale padding with resolution
#@markdown <br>

#@markdown ---
#@markdown <br>

#@markdown # [Advanced tweaking](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#advanced-tweaking) (optional) </h1>Just ignore all of this if you are new, or click the blue titles for instructions.
wav2lip_version = "Wav2Lip" # @param ["Wav2Lip", "Wav2Lip_GAN"]
nosmooth = True #@param {type:"boolean"}
#@markdown ### [Padding:](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#padding)</h1> (Up, Down, Left, Right) <br>
U = 0 #@param {type:"slider", min:-100, max:100, step:1}
D = 10 #@param {type:"slider", min:-100, max:100, step:1}
L = 0 #@param {type:"slider", min:-100, max:100, step:1}
R = 0 #@param {type:"slider", min:-100, max:100, step:1}
#@markdown <br>

#@markdown ### [Mask:](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#other-options)
size = 1.5 #@param {type:"slider", min:1, max:6, step:0.1}
feathering = 1 #@param {type:"slider", min:0, max:3, step:1}
mouth_tracking = False #@param {type:"boolean"}
debug_mask = False #@param {type:"boolean"}


#@markdown # [Other options:](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#other-options)

batch_process = False #@param {type:"boolean"}
output_suffix = "_Easy-Wav2Lip" #@param {type:"string"}
include_settings_in_suffix = False #@param {type:"boolean"}
preview_input = False #@param {type:"boolean"}
preview_settings = False #@param {type:"boolean"}
#@markdown preview_settings processes only one frame so you can see how it looks without doing the whole video
frame_to_preview = 100 # @param {type:"integer"}


import configparser

# Create a ConfigParser object
config = configparser.ConfigParser()

# Put all your variables in a dictionary
options = {
    'video_file': video_file,
    'vocal_file': vocal_file,
    'quality': quality,
    'output_height': output_height,
    'wav2lip_version': wav2lip_version,
    'use_previous_tracking_data': use_previous_tracking_data,
    'nosmooth': nosmooth
}
padding = {
    'U': U,
    'D': D,
    'L': L,
    'R': R
}
mask = {
    'size': size,
    'feathering': feathering,
    'mouth_tracking': mouth_tracking,
    'debug_mask': debug_mask
}
other = {
    'batch_process': batch_process,
    'output_suffix': output_suffix,
    'include_settings_in_suffix': include_settings_in_suffix,
    'preview_input': preview_input,
    'preview_settings': preview_settings,
    'frame_to_preview': frame_to_preview
}


# Add the dictionary to the ConfigParser object
config['OPTIONS'] = options
config['PADDING'] = padding
config['MASK'] = mask
config['OTHER'] = other

# Write the data to an INI file
with open('config.ini', 'w') as f:
    config.write(f)

!python run.py

from easy_functions import show_video
from IPython.display import Image
if preview_settings:
  if os.path.isfile(os.path.join('temp','preview.jpg')):
    display(Image(os.path.join('temp','preview.jpg')))
else:
  if os.path.isfile(os.path.join('temp','output.mp4')):
    print(f"Loading video preview...")
    show_video(os.path.join('temp','output.mp4'))

✅ degradations.py 已修复
Processing 视频剪辑25_爱给网_aigei_com.mp4 using 音频转码_爱给网_aigei_com.wav for audio
imports loaded!     
analysing audio...
612 frames to process
detecting face in every frame: 100%|██████████████████████████████| 612/612 [00:08<00:00, 76.34it/s]
mask size: 1.5, feathering: 1
Loading gfpgan
Starting...
Processing Wav2Lip: 100%|█████████████████████████████████████████| 612/612 [03:28<00:00,  2.93it/s]
converting to final video
视频剪辑25_爱给网_aigei_com_音频转码_爱给网_aigei_com successfully lip synced! It will be found here:
/content/视频剪辑25_爱给网_aigei_com_音频转码_爱给网_aigei_com_Easy-Wav2Lip.mp4
Execution time: 3m 55s
Loading video preview...
